In [251]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import make_scorer

In [195]:
census_df

,age AAGE,class of worker ACLSWKR,industry code ADTIND,occupation code ADTOCC,adjusted gross income AGI,education AHGA,wage per hour AHRSPAY,enrolled in edu inst last wk AHSCOL,marital status AMARITL,major industry code AMJIND,major occupation code AMJOCC,mace ARACE,hispanic Origin AREORGN,sex ASEX,member of a labor union AUNMEM,reason for unemployment AUNTYPE,full or part time employment stat AWKSTAT,capital gains CAPGAIN,capital losses CAPLOSS,divdends from stocks DIVVAL,federal income tax liability FEDTAX,tax filer status FILESTAT,region of previous residence GRINREG,state of previous residence GRINST,detailed household and family stat HHDFMX,detailed household summary in household HHDREL,instance weight MARSUPWT,migration code-change in msa MIGMTR1,migration code-change in reg MIGMTR3,migration code-move within reg MIGMTR4,live in this house 1 year ago MIGSAME,migration prev res in sunbelt MIGSUN,num persons worked for employer NOEMP,family members under 18 PARENT,total person earnings PEARNVAL,country of birth father PEFNTVTY,country of birth mother PEMNTVTY,country of birth self PENATVTY,citizenship PRCITSHP,total person income PTOTVAL,own business or self employed SEOTR,taxable income amount TAXINC,fill inc questionnaire for veteran's admin VETQVA,veterans benefits VETYN,weeks worked in year WKSWORK,Unnamed: 45
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.,NaN,NaN,NaN,NaN
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.,NaN,NaN,NaN,NaN
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.,NaN,NaN,NaN,NaN
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.,NaN,NaN,NaN,NaN
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [194]:
adult_df = pd.read_csv("adult.txt")
covtype_df = pd.read_csv("covtype.txt",names=list(range(0,55)))
letter_df = pd.read_csv("letter-recognition.txt",names=list(range(0,17))) 
census_df = pd.read_csv("census-income.txt")

In [145]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [115]:
imp_mean = SimpleImputer(missing_values=" ?", strategy="most_frequent").fit(adult_df)
adult_df = pd.DataFrame(imp_mean.transform(adult_df), columns=adult_df.columns)

# convert prediction data into binary labels
adult_df['dat'] = np.where(adult_df['dat'] == ' >50K',1,0)
adult_df.sample(15)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,dat
21567,39,Local-gov,164156,Assoc-acdm,12,Divorced,Other-service,Unmarried,White,Female,0,0,55,United-States,0
16498,52,Private,222107,Some-college,10,Divorced,Sales,Unmarried,White,Female,0,0,50,United-States,0
16381,20,Private,219835,Some-college,10,Never-married,Sales,Own-child,White,Male,0,0,30,United-States,0
26097,30,Private,224377,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,0
10122,50,Local-gov,259377,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,15024,0,40,United-States,1
25730,46,Private,249935,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,70,United-States,0
28370,32,Private,177675,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,0
13113,46,Local-gov,183168,Bachelors,13,Divorced,Prof-specialty,Unmarried,White,Female,0,0,43,United-States,0
24410,38,Private,146398,HS-grad,9,Never-married,Other-service,Unmarried,Black,Female,0,0,24,United-States,0
23125,31,Private,339482,1st-4th,2,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,Mexico,0


In [116]:
pred_column = covtype_df[54]
positive_label = pred_column.value_counts().index[0]

# convert prediction data into binary labels
pred_column = np.where(pred_column == positive_label, 1, 0)

covtype_df.drop(columns=[54],inplace=True)

In [117]:
scaler = MinMaxScaler().fit(covtype_df)
covtype_df = pd.DataFrame(scaler.transform(covtype_df),columns=covtype_df.columns)
covtype_df.insert(54,54,pred_column,True)
covtype_df.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
165202,0.568284,0.066667,0.075758,0.000000,0.223514,0.404665,0.854331,0.905512,0.586614,0.230169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
468529,0.802901,0.027778,0.227273,0.391553,0.373385,0.585078,0.795276,0.818898,0.555118,0.476927,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
193604,0.710355,0.816667,0.090909,0.312813,0.379845,0.728678,0.795276,0.940945,0.692913,0.249268,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
229231,0.672836,0.097222,0.151515,0.021475,0.222222,0.335535,0.862205,0.858268,0.527559,0.362610,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
367792,0.386193,0.597222,0.151515,0.301360,0.232558,0.229310,0.826772,0.984252,0.685039,0.252753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
406028,0.625313,0.472222,0.242424,0.042949,0.201550,0.413376,0.905512,0.964567,0.559055,0.153492,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
291188,0.295648,0.963889,0.303030,0.047960,0.242894,0.081073,0.696850,0.799213,0.633858,0.179562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
439838,0.746373,0.544444,0.303030,0.271296,0.288114,0.424617,0.830709,0.992126,0.657480,0.195873,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
233734,0.543272,0.538889,0.272727,0.257695,0.299742,0.246171,0.842520,0.992126,0.649606,0.249408,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
460913,0.599300,0.147222,0.348485,0.042949,0.244186,0.128425,0.889764,0.728346,0.322835,0.219155,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [118]:
letter_col,pred_column = letter_df[0], letter_df[16]
letter_df.drop(columns=[0,16],inplace=True)

In [119]:
scaler = MinMaxScaler().fit(letter_df)
letter_df = pd.DataFrame(scaler.transform(letter_df),columns=letter_df.columns)
letter_df.insert(0,0,letter_col,True)
letter_df.insert(16,16,pred_column,True)

# two ways to make binary classification labels
letter_df_p1 = np.where(letter_df[0] == 'O', 1, 0)
letter_df_p2 = np.where(letter_df[0].isin([chr(x) for x in range(ord('A'), ord('M') + 1)]), 1, 0)
letter_df.insert(17,17,letter_df_p1,True)
letter_df.insert(18,18,letter_df_p2,True)


letter_df.sample(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
9508,L,0.200000,0.466667,0.200000,0.333333,0.066667,0.066667,0.066667,0.333333,0.333333,0.000000,0.066667,0.400000,0.000000,0.533333,0.000000,8,0,1
16699,Y,0.066667,0.200000,0.133333,0.066667,0.066667,0.466667,0.666667,0.066667,0.333333,0.466667,0.733333,0.533333,0.066667,0.733333,0.066667,8,0,0
14115,K,0.400000,0.733333,0.600000,0.533333,0.466667,0.133333,0.600000,0.133333,0.466667,0.666667,0.800000,0.800000,0.333333,0.400000,0.266667,4,0,1
5535,A,0.266667,0.666667,0.400000,0.533333,0.200000,0.666667,0.200000,0.133333,0.200000,0.600000,0.066667,0.533333,0.133333,0.466667,0.266667,9,0,1
10657,H,0.266667,0.533333,0.333333,0.400000,0.333333,0.400000,0.466667,0.333333,0.266667,0.466667,0.333333,0.533333,0.200000,0.466667,0.400000,11,0,1
16796,K,0.533333,0.800000,0.466667,0.400000,0.266667,0.600000,0.533333,0.200000,0.466667,0.600000,0.266667,0.400000,0.400000,0.600000,0.266667,8,0,1
5925,E,0.200000,0.266667,0.333333,0.200000,0.133333,0.466667,0.533333,0.133333,0.533333,0.733333,0.400000,0.600000,0.133333,0.533333,0.266667,8,0,1
19246,W,0.200000,0.533333,0.333333,0.400000,0.266667,0.600000,0.666667,0.133333,0.133333,0.400000,0.533333,0.533333,0.466667,0.733333,0.066667,8,0,0
8889,H,0.333333,0.600000,0.333333,0.400000,0.133333,0.466667,0.466667,1.000000,0.000000,0.466667,0.400000,0.533333,0.200000,0.533333,0.000000,8,0,1
11634,N,0.333333,0.533333,0.466667,0.400000,0.266667,0.600000,0.466667,0.400000,0.400000,0.400000,0.400000,0.266667,0.400000,0.600000,0.133333,5,0,0


In [188]:
?census_df.dropna

In [189]:
census_df.replace([' Not in universe', ' ?'], np.nan, inplace = True) 
census_df.dropna(axis = 1, inplace = True)

In [190]:
census_df.sample(25)

,0,2,3,4,5,7,8,10,11,12,15,16,17,18,19,22,23,24,28,30,35,36,38,39,40,41
121182,43,45,23,Some college but no degree,0,Divorced,Other professional services,White,All other,Female,Children or Armed Forces,0,0,786,Single,Nonfamily householder,Householder,1740.15,Yes,1,Native- Born in the United States,0,2,52,94,- 50000.
151389,48,29,42,7th and 8th grade,0,Married-civilian spouse present,Transportation,White,All other,Male,Full-time schedules,0,0,0,Joint both under 65,Householder,Householder,1961.54,Not in universe under 1 year old,6,Native- Born in the United States,0,2,30,95,- 50000.
84287,4,0,0,Children,0,Never married,Not in universe or children,White,Mexican-American,Male,Children or Armed Forces,0,0,0,Nonfiler,Child <18 never marr not in subfamily,Child under 18 never married,1150.79,Yes,0,Native- Born in the United States,0,0,0,94,- 50000.
196845,27,35,26,Associates degree-occup /vocational,0,Never married,Finance insurance and real estate,White,All other,Female,Children or Armed Forces,0,0,0,Single,Nonfamily householder,Householder,2477.51,Yes,6,Native- Born in the United States,0,2,52,94,- 50000.
6537,45,33,13,High school graduate,0,Married-civilian spouse present,Retail trade,White,All other,Female,Children or Armed Forces,0,0,0,Joint both under 65,Spouse of householder,Spouse of householder,1106.37,Yes,6,Native- Born in the United States,0,2,52,94,- 50000.
77546,12,0,0,Children,0,Never married,Not in universe or children,White,All other,Male,Children or Armed Forces,0,0,0,Nonfiler,Child <18 never marr not in subfamily,Child under 18 never married,234.00,Not in universe under 1 year old,0,Native- Born in the United States,0,0,0,95,- 50000.
97574,43,41,33,Bachelors degree(BA AB BS),0,Married-civilian spouse present,Hospital services,White,All other,Male,Full-time schedules,0,0,0,Joint both under 65,Householder,Householder,1174.40,Not in universe under 1 year old,4,Native- Born in the United States,0,2,52,95,- 50000.
45400,9,0,0,Children,0,Never married,Not in universe or children,White,All other,Male,Children or Armed Forces,0,0,0,Nonfiler,Child <18 never marr not in subfamily,Child under 18 never married,1366.40,Yes,0,Native- Born in the United States,0,0,0,94,- 50000.
107221,61,0,0,Bachelors degree(BA AB BS),0,Married-civilian spouse present,Not in universe or children,White,All other,Female,Not in labor force,0,0,0,Joint both under 65,Spouse of householder,Spouse of householder,1303.48,Not in universe under 1 year old,0,Native- Born in the United States,0,2,0,95,- 50000.
195312,16,0,0,9th grade,0,Never married,Not in universe or children,White,All other,Female,Children or Armed Forces,0,0,0,Nonfiler,Other Rel <18 never marr not in subfamily,Other relative of householder,2628.23,No,0,Native- Born in the United States,0,2,0,94,- 50000.


In [191]:
census_df.iloc[0,13]

0

In [186]:
dftemp.isnull().sum()

0          0
1     100245
2          0
3          0
4          0
5          0
6     186943
7          0
8          0
9     100684
10         0
11         0
12         0
13    180459
14    193453
15         0
16         0
17         0
18         0
19         0
20    183750
21    184458
22         0
23         0
24         0
25    101212
26    101212
27    101212
28         0
29    183750
30         0
31    144232
32      6713
33      6119
34      3393
35         0
36         0
37    197539
38         0
39         0
40         0
41         0
dtype: int64

In [15]:
%%time
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


import warnings
# Create a pipeline - RF is a stand in, we will populate the classifier part below
pipe = Pipeline([('classifier', RandomForestClassifier())])
search_space = [{'classifier': [LogisticRegression(solver='saga')],
                 'classifier__penalty': ['none','l1','l2'],
                 'classifier__C': np.logspace(-8, 4, 13)},
                {'classifier': [RandomForestClassifier()],
                 'classifier__n_estimators': [10, 100, 1000],
                 'classifier__max_features': [1,2,6,12, 20]}]
# Create grid search 
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), verbose=0)
# Fit grid search
best_model = clf.fit(covtype_df.iloc[:5000,:-1], covtype_df.iloc[:5000,-1])

F:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", C

F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1321: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
F:\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", Converge

Wall time: 2min 6s


In [100]:
X, Y = covtype_df.iloc[:10000,:-1], covtype_df.iloc[:10000,-1]

In [101]:
%%time 
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# take all our penguin data, and reserve 50% of it for testing 
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.5,
                                                    random_state=0,
                                                    stratify=Y)


# Initializing Classifiers
clf1 = LogisticRegression(solver='saga',
                          random_state=0)
clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
clf3 = SVC(random_state=0)

clf4 = RandomForestClassifier(random_state=0)

clf5 = PassiveAggressiveClassifier(max_iter=1000, random_state=0, tol=1e-3)

# clf6 = OrthogonalMatchingPursuit()

# Building the pipelines
pipe1 = Pipeline([('classifier', clf1)])

pipe2 = Pipeline([('classifier', clf2)])

pipe3 = Pipeline([('classifier', clf3)])

pipe4 = Pipeline([('classifier', clf4)])

pipe5 = Pipeline([('classifier', clf5)])

# pipe6 = Pipeline([('classifier', clf6)])


# Setting up the parameter grids
param_grid1 = [{'classifier__penalty': ['none', 'l1', 'l2'],
                'classifier__C': np.logspace(-8, 4, 13)}]

param_grid2 = [{'classifier__n_neighbors': np.arange(1, 100, 10)}]
#                 'classifier__p': [1, 2]}]

param_grid3 = [{'classifier__kernel': ['rbf'],
                'classifier__C': np.power(10., np.arange(-4, 4)),
                'classifier__gamma': np.power(10., np.arange(-5, 0))},
               {'classifier__kernel': ['linear'],                
                'classifier__C': np.power(10., np.arange(-4, 4))}]

param_grid4 = [{'classifier__n_estimators': [1024],
                'classifier__max_features': [1,2,4,6,8,12,16,20]}]

param_grid5 = [{'classifier__C': np.logspace(-8,4,13),
               'classifier__loss': ['hinge', 'squared_hinge']}]

# param_grid6 = [{'classifier__n_nonzero_coefs': [.01, .05, .1, .2, .6]}]

# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3, param_grid4,param_grid5),
                            (pipe1, pipe2, pipe3, pipe4, pipe5),
                            ('Logistic', 'KNN', 'SVM', 'RF', 'PAC')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=2, # just 2-fold inner loop, i.e. train/test
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

Wall time: 21.9 ms


In [110]:
?accuracy_score

In [102]:
%%time 
# ^^ this handy Jupyter magic times the execution of the cell for you
warnings.filterwarnings('ignore')


cv_scores = {name: [] for name, gs_est in gridcvs.items()}

skfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=1)

# The outer loop for algorithm selection
c = 1
for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
    for name, gs_est in sorted(gridcvs.items()):
        print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

        # The inner loop for hyperparameter tuning
        gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
        y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
        acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
        print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gs_est.best_score_ * 100, acc * 100))
        cv_scores[name].append(acc)

    c += 1

    #FYI: This code uses X_train.iloc[... ] instead of X_train[...] because the 
    # penguin data is in a Dataframe instead of a numpy matrix


outer fold 1/5 | tuning KNN      | inner ACC 88.85% | outer ACC 88.70%
outer fold 1/5 | tuning Logistic | inner ACC 87.78% | outer ACC 86.80%
outer fold 1/5 | tuning PAC      | inner ACC 87.15% | outer ACC 86.50%
outer fold 1/5 | tuning RF       | inner ACC 90.62% | outer ACC 90.70%
outer fold 1/5 | tuning SVM      | inner ACC 88.70% | outer ACC 88.20%
outer fold 2/5 | tuning KNN      | inner ACC 88.15% | outer ACC 88.90%
outer fold 2/5 | tuning Logistic | inner ACC 87.48% | outer ACC 87.40%
outer fold 2/5 | tuning PAC      | inner ACC 87.13% | outer ACC 87.30%
outer fold 2/5 | tuning RF       | inner ACC 90.72% | outer ACC 90.70%
outer fold 2/5 | tuning SVM      | inner ACC 88.95% | outer ACC 89.30%
outer fold 3/5 | tuning KNN      | inner ACC 88.22% | outer ACC 88.80%
outer fold 3/5 | tuning Logistic | inner ACC 87.28% | outer ACC 88.00%
outer fold 3/5 | tuning PAC      | inner ACC 87.05% | outer ACC 87.30%
outer fold 3/5 | tuning RF       | inner ACC 89.85% | outer ACC 92.10%
outer 

In [103]:
# Looking at the results
for name in cv_scores:
    print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
          name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
print()
for name in cv_scores:
    print('{} best parameters'.format(name), gridcvs[name].best_params_)

Logistic | outer CV acc. 87.40% +\- 0.400
KNN      | outer CV acc. 89.12% +\- 0.471
SVM      | outer CV acc. 88.94% +\- 0.441
RF       | outer CV acc. 91.00% +\- 0.559
PAC      | outer CV acc. 87.00% +\- 0.310

Logistic best parameters {'classifier__C': 100.0, 'classifier__penalty': 'l1'}
KNN best parameters {'classifier__n_neighbors': 11}
SVM best parameters {'classifier__C': 1000.0, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf'}
RF best parameters {'classifier__max_features': 20, 'classifier__n_estimators': 1024}
PAC best parameters {'classifier__C': 0.1, 'classifier__loss': 'hinge'}


In [104]:
# Fitting a model to the whole training set
# using the "best" algorithm
best_algo = gridcvs['KNN']

best_algo.fit(X_train, y_train)
train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))

print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['SVM'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

Accuracy 88.62% (average over CV test folds)
Best Parameters: {'classifier__C': 1000.0, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf'}
Training Accuracy: 100.00%
Test Accuracy: 89.94%


In [105]:
X, Y = letter_df.iloc[:10000,1:-2], letter_df.iloc[:10000,-2]

In [106]:
%%time 
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# take all our penguin data, and reserve 50% of it for testing 
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.5,
                                                    random_state=0,
                                                    stratify=Y)


# Initializing Classifiers
clf1 = LogisticRegression(solver='saga',
                          random_state=0)

clf2 = KNeighborsClassifier(algorithm='ball_tree',
                            leaf_size=50)
clf3 = SVC(random_state=0)

clf4 = RandomForestClassifier(random_state=0)

clf5 = PassiveAggressiveClassifier(max_iter=1000, random_state=0, tol=1e-3)

# clf6 = OrthogonalMatchingPursuit()

# Building the pipelines
pipe1 = Pipeline([('classifier', clf1)])

pipe2 = Pipeline([('classifier', clf2)])

pipe3 = Pipeline([('classifier', clf3)])

pipe4 = Pipeline([('classifier', clf4)])

pipe5 = Pipeline([('classifier', clf5)])

# pipe6 = Pipeline([('classifier', clf6)])


# Setting up the parameter grids
param_grid1 = [{'classifier__penalty': ['none', 'l1', 'l2'],
                'classifier__C': np.logspace(-8, 4, 13)}]

param_grid2 = [{'classifier__n_neighbors': np.arange(1, 100, 10)}]
#                 'classifier__p': [1, 2]}]

param_grid3 = [{'classifier__kernel': ['rbf'],
                'classifier__C': np.power(10., np.arange(-4, 4)),
                'classifier__gamma': np.power(10., np.arange(-5, 0))},
               {'classifier__kernel': ['linear'],                
                'classifier__C': np.power(10., np.arange(-4, 4))}]

param_grid4 = [{'classifier__n_estimators': [1024],
                'classifier__max_features': [1,2,4,6,8,12,16,20]}]

param_grid5 = [{'classifier__C': np.logspace(-8,4,13),
               'classifier__loss': ['hinge', 'squared_hinge']}]

# param_grid6 = [{'classifier__n_nonzero_coefs': [.01, .05, .1, .2, .6]}]

# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3, param_grid4,param_grid5),
                            (pipe1, pipe2, pipe3, pipe4, pipe5),
                            ('Logistic', 'KNN', 'SVM', 'RF', 'PAC')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       scoring='accuracy',
                       n_jobs=1,
                       cv=2, # just 2-fold inner loop, i.e. train/test
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

Wall time: 17.9 ms


In [107]:
%%time 
# ^^ this handy Jupyter magic times the execution of the cell for you
warnings.filterwarnings('ignore')


cv_scores = {name: [] for name, gs_est in gridcvs.items()}

skfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=1)

# The outer loop for algorithm selection
c = 1
for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
    for name, gs_est in sorted(gridcvs.items()):
        print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

        # The inner loop for hyperparameter tuning
        gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
        y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
        acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
        print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gs_est.best_score_ * 100, acc * 100))
        cv_scores[name].append(acc)

    c += 1

    #FYI: This code uses X_train.iloc[... ] instead of X_train[...] because the 
    # penguin data is in a Dataframe instead of a numpy matrix


outer fold 1/5 | tuning KNN      | inner ACC 97.78% | outer ACC 98.60%
outer fold 1/5 | tuning Logistic | inner ACC 96.20% | outer ACC 96.20%
outer fold 1/5 | tuning PAC      | inner ACC 96.20% | outer ACC 96.20%
outer fold 1/5 | tuning RF       | inner ACC 98.38% | outer ACC 98.90%
outer fold 1/5 | tuning SVM      | inner ACC 97.97% | outer ACC 98.60%
outer fold 2/5 | tuning KNN      | inner ACC 98.23% | outer ACC 98.60%
outer fold 2/5 | tuning Logistic | inner ACC 96.20% | outer ACC 96.20%
outer fold 2/5 | tuning PAC      | inner ACC 96.20% | outer ACC 96.20%
outer fold 2/5 | tuning RF       | inner ACC 98.40% | outer ACC 98.50%
outer fold 2/5 | tuning SVM      | inner ACC 98.50% | outer ACC 98.10%
outer fold 3/5 | tuning KNN      | inner ACC 97.85% | outer ACC 98.30%
outer fold 3/5 | tuning Logistic | inner ACC 96.20% | outer ACC 96.20%
outer fold 3/5 | tuning PAC      | inner ACC 96.20% | outer ACC 96.20%
outer fold 3/5 | tuning RF       | inner ACC 98.38% | outer ACC 98.30%
outer 

In [108]:
# Looking at the results
for name in cv_scores:
    print('%-8s | outer CV acc. %.2f%% +\- %.3f' % (
          name, 100 * np.mean(cv_scores[name]), 100 * np.std(cv_scores[name])))
print()
for name in cv_scores:
    print('{} best parameters'.format(name), gridcvs[name].best_params_)

Logistic | outer CV acc. 96.20% +\- 0.000
KNN      | outer CV acc. 98.68% +\- 0.264
SVM      | outer CV acc. 98.48% +\- 0.204
RF       | outer CV acc. 98.56% +\- 0.215
PAC      | outer CV acc. 96.20% +\- 0.000

Logistic best parameters {'classifier__C': 1e-08, 'classifier__penalty': 'none'}
KNN best parameters {'classifier__n_neighbors': 1}
SVM best parameters {'classifier__C': 1000.0, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf'}
RF best parameters {'classifier__max_features': 6, 'classifier__n_estimators': 1024}
PAC best parameters {'classifier__C': 1e-08, 'classifier__loss': 'hinge'}


In [109]:
# Fitting a model to the whole training set
# using the "best" algorithm
best_algo = gridcvs['KNN']

best_algo.fit(X_train, y_train)
train_acc = accuracy_score(y_true=y_train, y_pred=best_algo.predict(X_train))
test_acc = accuracy_score(y_true=y_test, y_pred=best_algo.predict(X_test))

print('Accuracy %.2f%% (average over CV test folds)' %
      (100 * best_algo.best_score_))
print('Best Parameters: %s' % gridcvs['SVM'].best_params_)
print('Training Accuracy: %.2f%%' % (100 * train_acc))
print('Test Accuracy: %.2f%%' % (100 * test_acc))

Accuracy 98.14% (average over CV test folds)
Best Parameters: {'classifier__C': 1000.0, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf'}
Training Accuracy: 100.00%
Test Accuracy: 98.58%


In [111]:
best_algo.cv_results_

{'mean_fit_time': array([0.00999248, 0.00897574, 0.00997722, 0.00846028, 0.01097262,
        0.01098752, 0.00797904, 0.00797808, 0.01346684, 0.00997305]),
 'std_fit_time': array([1.98352337e-03, 9.96589661e-04, 1.54972076e-06, 5.14507294e-04,
        1.99687481e-03, 9.82999802e-04, 9.96708870e-04, 1.19209290e-07,
        3.46350670e-03, 9.97543335e-04]),
 'mean_score_time': array([0.16227698, 0.26413703, 0.28124559, 0.25383055, 0.24334741,
        0.2228924 , 0.30595279, 0.27277231, 0.36454248, 0.28488588]),
 'std_score_time': array([0.0022912 , 0.00365424, 0.04188406, 0.01446545, 0.01396132,
        0.00147712, 0.05162477, 0.00349283, 0.03889847, 0.01658368]),
 'param_classifier__n_neighbors': masked_array(data=[1, 11, 21, 31, 41, 51, 61, 71, 81, 91],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'params': [{'classifier__n_neighbors': 1},
  {'classifier__n_neighbors': 

In [22]:
len([covtype_df.iloc[5000:,:-1], covtype_df.iloc[5000:,-1]])

2

In [37]:
a = best_model.best_params_['classifier'].fit(covtype_df.iloc[:5000,:-1], covtype_df.iloc[:5000,-1]).predict(covtype_df.iloc[5000:,:-1])

In [35]:
np.mean(a)

0.0

In [16]:
best_model.best_params_

AttributeError: 'dict' object has no attribute 'predict'

In [9]:
# parameters = {'C':[1],'penalty':['l2']}
X = covtype_df.drop(columns=[0])
y = np.where(covtype_df[16] == 2,1,0)
log_reg = LogisticRegression()
clf = GridSearchCV(estimator=log_reg,param_grid=parameters)
clf.fit(X,y)
print(clf.score(X))

NameError: name 'parameters' is not defined

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score

X = adult_df[['workclass', 'marital-status', 'occupation', 'relationship']]
y = np.where(adult_df.iloc[:,-1] == ' >50K',1,0)

column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'),
                                        ['workclass', 'marital-status', 'occupation']),
                                      (OrdinalEncoder(), ['relationship']),
                                      remainder='drop')
logreg = LogisticRegression()
pipe = make_pipeline(column_trans, logreg)
cross_val_score(pipe, X, y, cv=10, scoring='f1')


In [ ]:
for i in range 3: 
    run_classifiers()

In [ ]:
def run_classifiers(data):
    iterations = 3
    for i in range(iterations):
        # draw 5k samples for training data, and set aside the rest for testing 
        X_train, Y_train, X_test, Y_test = draw_samples(data)
        # returns the gridsearchCV model list thing
        gridcvs = create_gridsearch()
        
        select_best_and_fit()
        
        get_statistics()
        
        send_brain_to_file()

In [206]:
def draw_samples(data, n = 5000):
    train_index = random.sample(range(0,len(data)), n)
    # assumes target column is last column
    X_train, Y_train = data.iloc[train_index, :-1], data.iloc[train_index, -1]
    
    test = data[~data.index.isin(train_index)]
    X_test, Y_test = test.iloc[:, :-1], test.iloc[:, -1]

    return X_train, Y_train, X_test, Y_test

In [224]:
?np.geomspace

array([1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00,
       1.e+01, 1.e+02, 1.e+03])

In [217]:
np.logspace(1,500,25)

array([1.00000000e+001, 6.18965819e+021, 3.83118685e+042, 2.37137371e+063,
       1.46779927e+084, 9.08517576e+104, 5.62341325e+125, 3.48070059e+146,
       2.15443469e+167, 1.33352143e+188, 8.25404185e+208, 5.10896977e+229,
       3.16227766e+250, 1.95734178e+271, 1.21152766e+292,             inf,
                   inf,             inf,             inf,             inf,
                   inf,             inf,             inf,             inf,
                   inf])

In [256]:
def create_gridsearch():
    scoring = {'accuracy' : 'accuracy',
               'f1' : 'f1', 
               'roc_auc' : 'roc_auc',
               'MCC' : make_scorer(matthews_corrcoef)}
    
    # Initializing Classifiers
    clf1 = LogisticRegression(solver='saga',
                              random_state=0)

    clf2 = KNeighborsClassifier(algorithm='ball_tree',
                                leaf_size=50)
    clf3 = SVC(random_state=0)

    clf4 = RandomForestClassifier(random_state=0)

    clf5 = PassiveAggressiveClassifier(max_iter=1000, random_state=0, tol=1e-3)


    # Building the pipelines
    pipe1 = Pipeline([('classifier', clf1)])

    pipe2 = Pipeline([('classifier', clf2)])

    pipe3 = Pipeline([('classifier', clf3)])

    pipe4 = Pipeline([('classifier', clf4)])

    pipe5 = Pipeline([('classifier', clf5)])



    # Setting up the parameter grids
    param_grid1 = [{'classifier__penalty': ['none', 'l1', 'l2'],
                    'classifier__C': np.logspace(-8, 4, 13)}]

    param_grid2 = [{'classifier__n_neighbors': np.geomspace(1, 500, num=25, dtype=int),
                    'classifier__weights': ['uniform', 'distance']}]

    param_grid3 = [{'classifier__kernel': ['rbf'],
                    'classifier__C': np.power(10., np.arange(-7, 4)),
                    'classifier__gamma': [0.001,0.005,0.01,0.05,0.1,0.5,1,2]},
                   {'classifier__kernel': ['linear'],                
                    'classifier__C': np.power(10., np.arange(-7, 4))},
                   {'classifier__kernel': ['polynomial'],
                    'classifier__degree': [2,3],
                    'classifier__C': np.power(10., np.arange(-7, 4))}]

    param_grid4 = [{'classifier__n_estimators': [1024],
                    'classifier__max_features': [1,2,4,6,8,12,16, 20]}]

    param_grid5 = [{'classifier__C': np.logspace(-8,4,13),
                    'classifier__loss': ['hinge', 'squared_hinge']}]


    # Setting up multiple GridSearchCV objects, 1 for each algorithm
    gridcvs = {}

    for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3, param_grid4,param_grid5),
                                (pipe1, pipe2, pipe3, pipe4, pipe5),
                                ('Logistic', 'KNN', 'SVM', 'RF', 'PAC')):
        gcv = GridSearchCV(estimator=est,
                           param_grid=pgrid,
                           scoring=scoring,
                           n_jobs=1,
                           cv=5, 
                           verbose=0,
                           refit=False)
        gridcvs[name] = gcv
    return gridcvs

In [257]:
gridcvs = create_gridsearch()

In [ ]:
run_gridsearch(gridcvs, X_train, y_train)

outer fold 1/5 | tuning KNN     

In [247]:
for name, gs_est in sorted(gridcvs.items()):
    print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

    # The inner loop for hyperparameter tuning
    gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
    y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
    acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
    print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
          (gs_est.best_score_ * 100, acc * 100))
    cv_scores[name].append(acc)
    print()

In [236]:
def run_gridsearch(gridcvs, X_train, Y_train):
    warnings.filterwarnings('ignore')
    
    cv_scores = {name: [] for name, gs_est in gridcvs.items()}

    skfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=1)

    # The outer loop for algorithm selection
    c = 1
    for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
        for name, gs_est in sorted(gridcvs.items()):
            print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

            # The inner loop for hyperparameter tuning
            gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
            y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
            acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
            print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
                  (gs_est.best_score_ * 100, acc * 100))
            cv_scores[name].append(acc)
            print()

        c += 1

In [ ]:
%%time 
# ^^ this handy Jupyter magic times the execution of the cell for you
warnings.filterwarnings('ignore')


cv_scores = {name: [] for name, gs_est in gridcvs.items()}

skfold = StratifiedKFold( n_splits=5, shuffle=True, random_state=1)

# The outer loop for algorithm selection
c = 1
for outer_train_idx, outer_valid_idx in skfold.split(X_train,y_train):
    for name, gs_est in sorted(gridcvs.items()):
        print('outer fold %d/5 | tuning %-8s' % (c, name), end='')

        # The inner loop for hyperparameter tuning
        gs_est.fit(X_train.iloc[outer_train_idx], y_train.iloc[outer_train_idx])
        y_pred = gs_est.predict(X_train.iloc[outer_valid_idx])
        acc = accuracy_score(y_true=y_train.iloc[outer_valid_idx], y_pred=y_pred)
        print(' | inner ACC %.2f%% | outer ACC %.2f%%' %
              (gs_est.best_score_ * 100, acc * 100))
        cv_scores[name].append(acc)

    c += 1

    #FYI: This code uses X_train.iloc[... ] instead of X_train[...] because the 
    # penguin data is in a Dataframe instead of a numpy matrix


In [215]:
?GridSearchCV